In [2]:
import pandas as pd

COL_NAME = 'School Name'
COL_DBN = 'School DBN'
COL_APPS = 'Grade 9 Total Applicants'
COL_TRUE_APPS = 'Grade 9 True Applicants'
COL_SEATS = 'Grade 9 Seats Available'
COL_OFFERS = 'Grade 9 Offers'

In [3]:
file_path = 'Data/fall-2023-admissions.xlsx'
df = pd.read_excel(file_path, sheet_name='School')

In [4]:
schools_to_exclude = ['03M485', '02M475', '05M692', '10X445', '10X696', '13K430', '14K449', '28Q687', '31R605']
columns = ['School District', COL_DBN, COL_NAME, COL_SEATS, COL_APPS, COL_TRUE_APPS, COL_OFFERS]

In [5]:
df_all = df[df['Category'] == 'All Students']
df_all = df_all[~ df_all['School DBN'].isin(schools_to_exclude)]
df_grade9 = df_all.dropna(subset=[COL_APPS])
df_grade9[COL_SEATS] = df_grade9[COL_SEATS].apply(lambda x: x.split(';')[0] if isinstance(x, str) else x).replace(['N/A', '0', 0], None)
df_grade9[COL_OFFERS] = df_grade9[COL_OFFERS].replace('N/A', None)

data = df_grade9[columns]

C:\Users\Tommaso\AppData\Local\Temp\ipykernel_12220\1874305094.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grade9[COL_SEATS] = df_grade9[COL_SEATS].apply(lambda x: x.split(';')[0] if isinstance(x, str) else x).replace(['N/A', '0', 0], None)
C:\Users\Tommaso\AppData\Local\Temp\ipykernel_12220\1874305094.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grade9[COL_OFFERS] = df_grade9[COL_OFFERS].replace('N/A', None)


In [6]:
import numpy as np

seed = 142
rng = np.random.default_rng(seed=seed)

def replace_suppressed(row):
	return row.apply(lambda x: rng.integers(low=1, high=5, endpoint=True) if x == 's' else x)

def replace_suppressed_by_diff(row):
	if row[COL_OFFERS] == 's^':
		row[COL_OFFERS] = row[COL_TRUE_APPS]
	return row

In [7]:
data = data.dropna(axis=0)

data = data.apply(replace_suppressed, axis=1)
data = data.apply(replace_suppressed_by_diff, axis=1)

data[COL_SEATS] = data[COL_SEATS].astype('int64')
data[COL_OFFERS] = data[COL_OFFERS].astype('int64')
data

,School District,School DBN,School Name,Grade 9 Seats Available,Grade 9 Total Applicants,Grade 9 True Applicants,Grade 9 Offers
775,1,01M292,Orchard Collegiate Academy,90,361,95,95
1308,1,01M448,University Neighborhood High School,105,1625,314,150
1533,1,01M450,East Side Community School,95,1011,333,109
1739,1,01M458,Forsyth Satellite Academy,28,59,6,6
1802,1,01M515,Lower East Side Preparatory High School,60,173,28,28
...,...,...,...,...,...,...,...
120263,32,32K545,EBC High School for Public Service - Bushwick,126,534,107,119
120342,32,32K549,The Brooklyn School for Social Justice,115,357,50,50
120413,32,32K552,The Academy of Urban Planning and Engineering,113,922,171,171
120571,32,32K554,All City Leadership Secondary School,62,963,291,63


In [8]:
export_dict = {}

for _, row in data.iterrows():
	name = row[COL_DBN] if pd.isna(row[COL_NAME]) else row[COL_NAME]
	dbn = row[COL_DBN]
	true_applicants = row[COL_TRUE_APPS]
	total_applicants = row[COL_APPS]
	capacity = row[COL_SEATS]
	# offers = row[COL_OFFERS]

	export_dict[name] = (dbn, capacity, true_applicants, total_applicants)

np.save('Data/schools_info.npy', export_dict, allow_pickle=True)
export_dict

{'Orchard Collegiate Academy': ('01M292', 90, 95, 361),
 'University Neighborhood High School': ('01M448', 105, 314, 1625),
 'East Side Community School': ('01M450', 95, 333, 1011),
 'Forsyth Satellite Academy': ('01M458', 28, 6, 59),
 'Lower East Side Preparatory High School': ('01M515', 60, 28, 173),
 'New Explorations into Science, Technology and Math High School': ('01M539',
  150,
  1842,
  3347),
 'Bard High School Early College': ('01M696', 125, 2602, 4801),
 '47 The American Sign Language and English Secondary School': ('02M047',
  69,
  73,
  385),
 'The Urban Assembly School for Emergency Management': ('02M135',
  115,
  48,
  334),
 'Stephen T. Mather Building Arts & Craftsmanship High School': ('02M139',
  108,
  96,
  472),
 'The Clinton School': ('02M260', 134, 1128, 2547),
 'Manhattan Early College School for Advertising': ('02M280', 112, 88, 482),
 'Urban Assembly Maker Academy': ('02M282', 125, 160, 1032),
 'Food and Finance High School': ('02M288', 150, 260, 1058),
 '